## PROJECT : FAKE NEWS CLASSIFIER (USING LSTM AND WORD_EMBEDDING)

OBJECTIVE: 

> We need to train our machine lerning model to categorize whether the NEWS is FAKE or not.

DATASET : https://www.kaggle.com/c/fake-news/data#

In [63]:
# importing the libraries 
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split


# tensorflow library
import tensorflow as tf
print (tf.__version__)
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

2.11.0


In [86]:
## GETTING THE DATASET :

df = pd.read_csv('/content/drive/MyDrive/UNIV.AI/NLP Intro /Datasets/FAKE NEWS DATASET/train.csv', usecols = ['title','text', 'label'])
display(df.shape, df.head())

(20800, 3)

,title,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1


In [87]:
# Removing all the rows having null values

df = df.dropna()

# # Resetting the indexes 
df.reset_index(drop = True, inplace = True)

display(df.shape, df.head())

(20203, 3)

,title,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1


In [94]:
# Spliting the Predictor and Label:
X = df['title']
y = df['label']

## CLEANING THE TEXTS AND PERFORMING WORD_EMBEDDING using Tensorflow

In [96]:
## CLEANING THE texts (X) / Preprocessing the Data
ps = PorterStemmer()
corpus = []
for i in range (len(X)):
  plain_text = re.sub('[^a-zA-Z]', ' ', X[i]) 
  plain_text = plain_text.lower()
  plain_text = plain_text.split()

  plain_text = [ps.stem(word) for word in plain_text if not word in stopwords.words('english')]
  corpus.append(" ".join(plain_text))


In [99]:
# Initializing the vocabulary size/ dictionary Size:

voc_size  = 5000

### One-hot reresentation

In [100]:
one_hot_rep = [one_hot(words, voc_size) for words in corpus]
print(one_hot_rep)

[[4584, 2633, 409, 3335, 1208, 2516, 1727, 4752, 791, 426], [4405, 1939, 92, 1752, 446, 4979, 2442], [4995, 2642, 4654, 4014], [1064, 2301, 1734, 1317, 4022, 3762], [2066, 446, 1513, 1957, 3865, 363, 446, 2583, 4089, 4082], [224, 1486, 2268, 3724, 589, 1648, 1205, 3661, 3314, 799, 4989, 4046, 865, 2307, 2442], [1354, 1354, 2714, 3831, 4921, 1242, 1920, 167, 4893, 1358, 1327, 1107], [881, 2881, 2504, 4390, 1047, 735, 3342, 1848, 517, 4014, 598], [231, 283, 3238, 4710, 1648, 4174, 2209, 4956, 4261, 1443, 517, 4014, 598], [4530, 4271, 592, 3241, 4690, 732, 1648, 1633, 517, 4014, 598], [1868, 1423, 3365, 2392, 1868, 48, 4021, 2957, 1648, 1251], [4111, 2344, 3211, 2598, 4918, 3049, 206, 3512], [850, 3911, 4748, 2658, 2400, 1857, 278, 747, 4425, 4222, 2568], [1317, 2152, 1208, 48, 1648, 4690], [175, 1342, 3264, 3519, 3716, 2473, 2165, 2869, 4431], [3357, 3487, 2300, 2443, 4380, 1402, 1880, 517, 4014, 598], [2318, 2877, 4675, 4967, 2657, 517, 4014, 598], [558, 4445, 475, 2466, 3671, 522, 4804

## Embedding Representation

In [102]:
sent_length  = 20
embedded_text  = pad_sequences(one_hot_rep, padding = 'post', maxlen = sent_length)
print (embedded_text[:5])

[[4584 2633  409 3335 1208 2516 1727 4752  791  426    0    0    0    0
     0    0    0    0    0    0]
 [4405 1939   92 1752  446 4979 2442    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [4995 2642 4654 4014    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [1064 2301 1734 1317 4022 3762    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [2066  446 1513 1957 3865  363  446 2583 4089 4082    0    0    0    0
     0    0    0    0    0    0]]


In [103]:
# initializing our Sequential model with 20 dimensions/features :
dim = 40

model = Sequential(name= 'LSTM_Embedded_Model')
model.add(Embedding(voc_size, dim, input_length = sent_length)) # Adding our embedding layer
model.add(LSTM(100)) # adding 1 LSTM Layer
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "LSTM_Embedded_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [104]:
x_final = np.array(embedded_text)
y_final = np.array(y)

In [105]:
display(x_final.shape, y_final.shape)

(20203, 20)

(20203,)

In [106]:
## Splitting the training and testing data:
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, train_size = 0.75, random_state =  41)

## Training the model:


In [107]:
# we already defined the model:
# Fitting the model with training data:

model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs= 10)

Epoch 1/10
474/474 [==============================] - 13s 23ms/step - loss: 0.2550 - accuracy: 0.8919 - val_loss: 0.1830 - val_accuracy: 0.9262
Epoch 2/10
474/474 [==============================] - 11s 22ms/step - loss: 0.1420 - accuracy: 0.9477 - val_loss: 0.1826 - val_accuracy: 0.9258
Epoch 3/10
474/474 [==============================] - 12s 25ms/step - loss: 0.1132 - accuracy: 0.9591 - val_loss: 0.1998 - val_accuracy: 0.9248
Epoch 4/10
474/474 [==============================] - 11s 23ms/step - loss: 0.0880 - accuracy: 0.9694 - val_loss: 0.2082 - val_accuracy: 0.9240
Epoch 5/10
474/474 [==============================] - 11s 24ms/step - loss: 0.0650 - accuracy: 0.9785 - val_loss: 0.2866 - val_accuracy: 0.9210
Epoch 6/10
474/474 [==============================] - 11s 23ms/step - loss: 0.0555 - accuracy: 0.9825 - val_loss: 0.2713 - val_accuracy: 0.9216
Epoch 7/10
474/474 [==============================] - 11s 22ms/step - loss: 0.0400 - accuracy: 0.9881 - val_loss: 0.2741 - val_accuracy:

## Performance Matrix and Accuracy

In [108]:
from sklearn.metrics  import accuracy_score , classification_report

y_pred = model.predict(x_test)

158/158 [==============================] - 1s 7ms/step


In [109]:
final_predictions = []
for i in y_pred:
  if (i[0]) > 0.5:
    final_predictions.append(1)
  else:
    final_predictions.append(0)

In [111]:
print (f'''
MODEL REPORTS:

1) Test Accuracy : {accuracy_score(y_test, final_predictions)}

2) Classification Report: 
{classification_report(y_test, final_predictions)}

''')


MODEL REPORTS:

1) Test Accuracy : 0.9164521876856068

2) Classification Report: 
              precision    recall  f1-score   support

           0       0.91      0.92      0.92      2589
           1       0.92      0.91      0.91      2462

    accuracy                           0.92      5051
   macro avg       0.92      0.92      0.92      5051
weighted avg       0.92      0.92      0.92      5051



